# RISM - A Gentle Introduction

We are going to write a Python implementation of the extended RISM equation with the goal of computing the direct, indirect and total correlation function along with the radial distribution function.

First, we import the libraries we will be needing:

In [ ]:
import numpy as np
from scipy.fftpack import dstn, idstn
from scipy.special import erf
import matplotlib.pyplot as plt

First, we need to set up our problem domain. While the RISM equations are written as continuous functions $r$ (e.g. $c(r)$), computationally we solve them on a discretised grid.

We'll set up a discrete domain of length (or radius since we are working with spherically symmetric functions) $r = 15\, Å$ partitioned into $N = 100$ points. The resulting grid spacing is given by $\Delta r = \frac{r}{N}$.

In [ ]:
r = 20.48
N = 1024
dr = r / N

We also want to set up a grid spacing for our Fourier-space grid such that $\Delta r \Delta k = \frac{\pi}{N}$

In [ ]:
dk = 2.0 * np.pi / (2.0 * N * dr)

Lets check the equality we stated above:

In [ ]:
np.isclose(dr * dk, np.pi / N)

We use `np.isclose()` here because we are comparing floating point numbers. Using `==` would not work since there is always some precision loss with floats. You need to compared them within tolerances instead.

We can use these to generate our real and Fourier-space midpoint grids.

In [ ]:
r_grid = np.arange(0.5, N, 1.0) * dr
k_grid = np.arange(0.5, N, 1.0) * dk

## Simple Argon Liquid

Now that we have our problem domain set up, we can focus on solving a particular problem. We're going to look at solving RISM for a simple Lennard-Jones Argon liquid system.

We first need to set up the thermodynamical state of the system and the unit systems we're going to use. We'll be working with energies in kJ / mol.

In [ ]:
T = 200.0 # Kelvin
kB = 8.314462618e-3 # kJ / mol / K
beta = 1.0 / T / kB # Thermodynamic Beta

We also need to define the parameters for Argon, as well as its number density. 

In [ ]:
ar_epsilon = 0.997 # kJ / mol
ar_sigma = 3.4 # Angstroms
ar_rho = 0.021017479720736955 # 1 / Angstroms^3

We need to tabulate the LJ potential with the above parameters. We define a function to evaluate the potential over a grid, and then we plot the potential.

In [ ]:
def LJ(epsilon, sigma, r):
    return 4.0 * epsilon * ( np.power( (sigma / r), 12) - np.power( (sigma / r), 6) )

In [ ]:
ar_ur = LJ(ar_epsilon, ar_sigma, r_grid)

plt.plot(r_grid, ar_ur)
plt.ylim([-1.5, 1.0])
plt.show()

One aspect of the RISM equation that we need to consider is the intramolecular correlation function $\omega(k)$. In the case of a simple liquid, $\omega(k) = 1$, in which case we the RISM equation reduces to the Ornstein-Zernike (OZ) equation.
We'll implement the full RISM equation, but we will set $\omega(k)$ to 1 for the case our current system.

In [ ]:
ar_wk = np.ones(N)

Another aspect is the closure. The closure is what relates our unknowns ($h(r)$ and $c(r)$), in a sense "closing" the RISM equation.
Generally the closure in terms of $c(r)$ is $$ c(r) = e^{-\beta u(r) + t(r) - B(r)} - t(r) - 1 $$ where $B(r)$ is the bridge function - something that we need to approximate - and $t(r) = h(r) - c(r)$ is the indirect correlation function.

We're going to use the simplest approximation, $B(r) = 0$. This is the Hypernetted Chain (HNC) closure.

$$ c(r) = e^{-\beta u(r) + t(r)} - t(r) - 1 $$

In [ ]:
def HNC(beta, ur, tr):
    return np.exp(-beta * ur + tr) - 1.0 - tr

The final detail before we implement the RISM equation is that we need to be able to transform our correlation functions into Fourier-space and back.
We use the Fourier transforms of spherically symmetric functions:

$$ f(k) = 4 \pi \int_0^\infty f(r) r^2 \frac{\sin(kr)}{kr} dr $$

and the inverse:

$$ f(r) = \frac{1}{2 \pi^2} \int_0^\infty f(k) k^2 \frac{\sin(kr)}{kr} dk $$

The function $J_0(kr) = \frac{\sin(kr)}{kr}$ is the zeroth-order spherical Bessel function of the first kind, so sometimes these transforms are called Fourier-Bessel (or Hankel) transforms.

This can neatly be related to the Fourier sine transform:

$$ f(k) = \int_0^\infty f(r) \sin(kr) dr $$

$$ f(r) = \frac{2}{\pi} \int_0^\infty f(k) \sin(kr) dk $$

So that the fourier transform of the spherically symmetric function $f(r)$ is just the Fourier sine transform of the function $f(r) \cdot r$

Scipy has discrete sine transform functions we can use.

In [ ]:
def fbt(fr, r, k, dr):
    return 2.0 * np.pi * dr * dstn(fr * r, type=4, axes=[0]) / k

def ifbt(fk, r, k, dk):
    return dk / 4.0 / np.pi / np.pi * idstn(fk * k, type=4, axes=[0]) / r

We have all the key parts to solve the RISM equation now.
The equation in Fourier-space is $$h(k) = \omega(k) * c(k) * \omega(k) - \rho * \omega(k) * c(k) * h(k)$$
We can rearrange this for $h(k)$ $$h(k) = [1 - \rho * \omega(k) * c(k)]^{-1} \omega(k) * c(k) * \omega(k)$$

For brevity, I'm going to drop the function argument $(k)$
$$h = [1 - \rho\omega c]^{-1} \omega c \omega$$

We're interested in calculating the indirect correlation function $t(k)$ $$t = [1 - \rho\omega c]^{-1} \omega c \omega - c$$

In [ ]:
def RISM(cr, w, p, r, k, dr, dk):
    # Transform c(r) to c(k)
    ck = fbt(cr, r, k, dr)
    # Solve the RISM equation for t(k)
    tk = w * ck * w / (1.0 - p * w * ck) - ck
    # Transform t(k) to t(r)
    tr = ifbt(tk, r, k, dk)
    return tr

We have all the equations we need to perform a single closure and RISM calculation. Now we need to numerically solve this problem.

The RISM equation is an integro-differential equation, and is posed to be solved using fixed-point iteration (i.e. the self-consistent approach used in electronic structure theory e.g. for solving Hartree-Fock).

We set some initial guess for $t_0(r)$, and use that as input into the closure and RISM equation, and get a new value out. If we represent the closure and RISM equation as an operator $F(\cdot)$ on $t(r)$ then we get a value of $F(t(r))$.

We can then mix this new value with our initial value to obtain a new solution for $t(r)$ $$t_1(r) = \alpha F(t_0(r))  + (1 - \alpha) t_0(r) $$

More generally $$t_{k+1}(r) = \alpha F(t_k(r)) + (1 + \alpha) t_k(r)$$

where $\alpha$ is a dampening parameter. The lower this is, the stiffer our iteration process is (because we're including more of the previous solution than the current). This is called Picard iteration.


We keep iterating this set of equations and check the difference between current and previous solutions until that difference is below some tolerance $\zeta$ $$ |t_{k+1}(r) - t_{k}(r)| < \zeta $$

Once the difference is below this tolerance, the calculation is finished.

We use an initial guess of $t(r) = 0$ and a tolerance of $\zeta = 1\times10^{-5}$

In [ ]:
# Setting up parameters for iterations
tolerance = 1e-5
max_steps = 5000
step = 0

# 1.0 is essentially undamped direct iteration
alpha = 1.0

# Setting up initial guess
tr = np.zeros(N)

while step < max_steps:
    # Store previous t(r)
    tr_prev = tr

    # t(r) -> c(r) from closure
    cr = HNC(beta, ar_ur, tr)

    # c(r) -> F(t(r)) from RISM equation
    f_tr = RISM(cr, ar_wk, ar_rho, r_grid, k_grid, dr, dk)

    # Use Picard iteration to get the new solution
    tr_new = alpha * f_tr + (1.0 - alpha) * tr_prev
    
    # Check the absolute value of the difference between the current and previous solution
    # We use .max() because we want to check that all values are below the tolerance, and
    # if the maximum value of the difference array is below, then so must the other values be.
    diff = np.sqrt(np.power((tr_new - tr_prev), 2).sum() * dr)
    
    # Set t(r) to the new t(r) we just computed
    tr = tr_new

    # Print the iteration step and difference every 10 steps
    if step % 10 == 0:
        print("Iteration: {step} Diff: {diff:.2e}".format(step=step, diff=diff))

    # Test if the difference is below our tolerance
    if diff < tolerance:
        # Print the final step
        print("Final Iteration: {step} Diff: {diff:.2e}".format(step=step, diff=diff))
        break

    # Update our step counter. We have a max_step because we don't want this calculation to go 
    # on forever and freeze our PC.
    step += 1

Once solved, we can use the calculated correlation functions to calculate the radial distribution function and plot it.

In [ ]:
# Get c(r) from the solved t(r)
cr = HNC(beta, ar_ur, tr)

# Get h(r)
hr = tr + cr

# Get and plot g(r)
gr = hr + 1

plt.plot(r_grid, gr)
plt.show()

This is the radial distribution function for a Lennard-Jones Argon liquid!

On your machine you might find that it took over 300 iterations to come to a solution.
We're using a very simple iteration scheme, but there are more complex iteration schemes we can use that can reduce the number of iterations.
We will look at one convergence acceleration method later, but most methods are beyond the scope of this tutorial.

## Nitrogen - Introducing Molecular Liquids and the Coulomb Potential

We want to look at more than just simple Lennard-Jones fluids. We can consider an uncharged homonuclear diatomic molecule, and nitrogen is a good such example.
Lets define the parameters, system and units.

In [ ]:
T = 72.0 # Kelvin
kB = 8.314462618e-3 # kJ / mol / K
beta = 1.0 / T / kB # Thermodynamic Beta

n2_epsilon = 0.3658363552 # kJ / mol
n2_sigma = 3.341 # Angstroms
n2_rho = 0.01867 # 1 / Angstroms^3

Notice that we've defined a bond length here. We need this for computing the $\omega(k)$.

First we need to define the shape of our arrays. We're still working $N$ grid points but we're now dealing with 2 atoms (or sites in the terminology of RISM).

It's useful to define a variable for our system size.

In [ ]:
ns = 2 # Number of sites

n2_wk = np.zeros((N, ns, ns)) # 3D array of number of points X number of sites X number of sites

Now we need to define what the intramolecular correlation function looks like $$\omega_{ij}(k) = \delta_{ij} + (1 - \delta_{ij})\frac{\sin(kd_{ij})}{kd_{ij}}$$
where $\delta_{ij}$ is the Kronecker delta, and $d_{ij}$ is the distance between site $i$ and $j$.


For nitrogen, we can explicitly define these for each $ij$ pair. If we number both nitrogens as $\textrm{N}_1$ and $\textrm{N}_2$ and we state the bond length between them to be $1.1\, Å$ then we can write 

$$\omega_{\textrm{N}_1 \textrm{N}_1} = \omega_{\textrm{N}_2 \textrm{N}_2} = 1$$

$$\omega_{\textrm{N}_1 \textrm{N}_2} = \omega_{\textrm{N}_2 \textrm{N}_1} = \frac{\sin(1.1k)}{1.1k}$$

In [ ]:
n2_bond_length = 1.1

# [:, 0, 0] indicates a slice over the first axis (which we defined as the number of points above, and indexing the last 2 axes explicitly with 0 and 0 (or 1 and 1) etc.
n2_wk[:, 0, 0] = n2_wk[:, 1, 1] = np.ones(N)
n2_wk[:, 0, 1] = n2_wk[:, 1, 0] = np.sin(k_grid * n2_bond_length) / k_grid / n2_bond_length

We can compute the potential in a similar fashion

In [ ]:
n2_ur = np.zeros((N, ns, ns))

n2_ur[:, 0, 0] = n2_ur[:, 0, 1] = n2_ur[:, 1, 0] = n2_ur[:, 1, 1] = LJ(n2_epsilon, n2_sigma, r_grid)

We can do it like this because the two interaction sites are essentially equivalent, they have the exact same LJ parameters and charge, even if they are in different positions in space. We've accounted for the spatial difference with $\omega(k)$.

In [ ]:
plt.plot(r_grid, n2_ur[:, 0, 0])
plt.ylim([-1.5, 1.0])
plt.show()

We have our inputs (the potential energy and the intramolecular correlation function), so lets solve the RISM equation.

We're going to actually wrap a solver in a function for easier use in the future.

In [ ]:
def picard_iteration(tolerance, max_step, alpha, initial_tr, beta, ur, wk, rho):
    # Set up our loop variable tr
    tr = initial_tr
    
    # Define our counter
    step = 0
    
    while True:
        # Store previous t(r)
        tr_prev = tr

        # t(r) -> c(r) from closure
        cr = HNC(beta, ur, tr)

        # c(r) -> F(t(r)) from RISM equation
        f_tr = RISM(cr, wk, rho, r_grid, k_grid, dr, dk)

        # Use Picard iteration to get the new solution
        # Since we set alpha = 1.0 above, this is 
        # essentially direct iteration (tr_new = f_tr).
        tr_new = alpha * f_tr + (1.0 - alpha) * tr_prev

        # Check the absolute value of the difference between the current and previous solution
        # We use .max() because we want to check that all values are below the tolerance, and
        # if the maximum value of the difference array is below, then so must the other values be.
        diff = np.sqrt(np.power((tr_new - tr_prev), 2).sum() * dr)

        # Set t(r) to the new t(r) we just computed
        tr = tr_new

        # Print the iteration step every 100 steps and difference
        if step % 100 == 0:
            print("Iteration: {step} Diff: {diff:.2e}".format(step=step, diff=diff))

        # Test if the difference is below our tolerance
        if diff < tolerance:
            print("Final Iteration: {step} Diff: {diff:.2e}".format(step=step, diff=diff))
            break
        
        # If we reach max_step, raise an error!
        if step == max_step:
            print("Reached max steps!")
            break

        # Update our step counter. We have a max_step because we don't want this calculation to go 
        # on forever and freeze our PC.
        step += 1
    
    # Return solved t(r)
    return tr

Before we start the solver, there's something else we need to consider - the fbt and ifbt functions.

We wrote them for 1D arrays in mind, but we're working with 3D arrays now. If you were try to use the above function without modifying the fbt and ifbt functions, you'd see that it wouldn't work because the r and k grids are not 3D, they're 1D, but our fr and fk grids are 3D now.

We need to refactor them for 3D arrays. Fortunately this is an easy fix, we just need to introduce a new axis to the r and k grid. These are just empty axes that Numpy uses to properly broadcast a 1D array over 3D arrays.

In [ ]:
def fbt(fr, r, k, dr):
    return 2.0 * np.pi * dr * dstn(fr * r[:, np.newaxis, np.newaxis], type=4, axes=[0]) / k[:, np.newaxis, np.newaxis]

def ifbt(fk, r, k, dk):
    return dk / 4.0 / np.pi / np.pi * idstn(fk * k[:, np.newaxis, np.newaxis], type=4, axes=[0]) / r[:, np.newaxis, np.newaxis]

We also have a slightly more complicated problem on our hands now. Using an initial guess of 0 might not cut it, so we'll try using the Mayer-f function as an initial guess

$$ f(r) = e^{-\beta u(r)} - 1 $$

In [ ]:
tolerance = 1e-8
max_step = 1000
alpha = 0.5
initial_tr = np.exp(-beta * n2_ur) - 1
tr = picard_iteration(tolerance, max_step, alpha, initial_tr, beta, n2_ur, n2_wk, n2_rho)

1000 iterations doesn't seem to cut it on my machine. We might need to introduce a new scheme to help with convergence.

We're going to introduce charging, which is where we slowly switch on the potential and run the solver at each stage.

In [ ]:
tolerance = 1e-8
max_step = 1000
alpha = 0.5
initial_tr = np.zeros((N, ns, ns))
lambdas = 10

for ilam in range(1, lambdas+1):
    lam = ilam / lambdas
    print(lam)
    tr = picard_iteration(tolerance, max_step, alpha, initial_tr, beta, lam * n2_ur, n2_wk, n2_rho)
    initial_tr = tr

plt.plot(r_grid, tr[:, 0, 0] + HNC(beta, n2_ur, tr)[:, 0, 0] + 1.0)
plt.show()

1000 iterations doesn't seem to cut it on my machine. We might need to implement a different solver.

Lets take a look at a simple convergence acceleration method called Ng acceleration.

Convergence acceleration methods are a family of numerical methods that use the history of solutions to guide towards a new solution quickly.

The basic idea of the Ng method is to save the previous 3 solutions and to use those to parametrise a new solution.

Lets define a few things for bookkeeping:

$$ f_n = t_k(r) $$

$$ g_n = F(t_k(r)) $$

$$ d_n = g_n - f_n = F(t_k(r)) - t_k(r) $$

If we save $n=3$ solutions, we have the variables $f_{n-2}$, $g_{n-2}$, $f_{n-1}$, $g_{n-1}$, $f_{n}$, and $g_{n}$ available to us. This is our history of solutions, and we will use these to guide our solver to a new solution.

$d_n$ is essentially a residual of our system. We construct a linear system of equations to solve for coefficients that will be used to construct our new solution.

We define further the following residuals:

$$d_{01} = d_n - d_{n-1}$$

$$d_{02} = d_n - d_{n-2}$$

Then we construct a matrix A

$$
A = 
\begin{bmatrix}
\langle d_{01}, d_{01} \rangle & \langle d_{01}, d_{02} \rangle \\
\langle d_{02}, d_{01} \rangle & \langle d_{02}, d_{02} \rangle 
\end{bmatrix}
$$

and a vector b

$$
b = 
\begin{bmatrix}
\langle d_{n}, d_{01} \rangle \\
\langle d_{n}, d_{02} \rangle
\end{bmatrix}
$$

where $\langle \cdot, \cdot \rangle$ is an inner (or dot) product.

We can then construct a linear system $Ax = b$ where we solve for coefficients $x = x_1, x_2$. These coefficient can be used to construct a new solution.

$$ f_{n+1} = (1 - x_1 - x_2) g_n + x_1 g_{n-1} + x_2 g_{n-2} $$

where $f_{n+1} = t_{k+1}(r)$.

We test for convergence using the L2-norm $$ \left\Vert d_n \right\Vert = \left\{\int \left|d_n \right|^2 dr\right\}^{\frac{1}{2}} $$

Lets write a simple function for a single Ng step.

In [ ]:
def Ng_step(f, g):
    # f and g are our lists of f and g functions
    # We'll index relative to n = 2, so that we can better match the maths (2 rather than 3 because Python is a zero-indexed language)
    n = 2
    
    # First, preserve the shape of the inputs, since we'll be flattening them into vectors
    dims = f[0].shape
    
    # Compute the residuals. We need to flatten these into vectors.
    d_n = (g[n] - f[n])
    d_1 = (g[n-1] - f[n-1])
    d_2 = (g[n-2] - f[n-2])
    
    d01 = d_n - d_1
    d02 = d_n - d_2
    
    # We can fill up A and b now
    A = np.zeros((2, 2))
    b = np.zeros(2)
    
    A[0, 0] = np.inner(d01, d01)
    A[0, 1] = np.inner(d01, d02)
    A[1, 0] = np.inner(d01, d02)
    A[1, 1] = np.inner(d02, d02)
    
    b[0] = np.inner(d_n, d01)
    b[1] = np.inner(d_n, d02)
    
    # We can now perform a linear solve to get x1 and x2
    x = np.linalg.solve(A, b)
    
    # And we can construct a new solution
    f_new = (1.0 - x[0] - x[1]) * g[n] 
    + x[0] * g[n-1] 
    + x[1] * g[n-2]
    
    return f_new.reshape(dims)

And lets write a function to do perform the full iterative procedure

In [ ]:
def Ng_iteration(tolerance, max_step, alpha, initial_tr, beta, ur, wk, rho):
    # Set up our loop variable tr
    tr = initial_tr
    
    # Define our counter
    step = 0
    
    # Define empty f and g lists
    f = []
    g = []
    
    while True:
        # Store previous t(r)
        tr_prev = tr

        # t(r) -> c(r) from closure
        cr = HNC(beta, ur, tr)

        # c(r) -> F(t(r)) from RISM equation
        f_tr = RISM(cr, wk, rho, r_grid, k_grid, dr, dk)

        # Use Picard iteration to get the new solution
        # if we've not got a history of 3 solutions yet
        if len(f) < 2:
            
            # Update f and g lists.
            f.append(tr)
            g.append(f_tr)
            
            tr_new = alpha * f_tr + (1.0 - alpha) * tr_prev
            
        # Otherwise lets use the Ng method
        else:
            
            # Update f and g lists
            f.append(tr)
            g.append(f_tr)
            
            tr_new = Ng_step(f, g)
            
            # We want to keep the list to 3 elements, so we pop off the oldest elements
            f.pop(0)
            g.pop(0)
            
        # Check the convergence criterion
        diff = np.sqrt(np.power((tr_new - tr_prev), 2).sum() * dr)

        # Set t(r) to the new t(r) we just computed
        tr = tr_new

        # Print the iteration step every 10 steps and difference
        if step % 10 == 0:
            print("Iteration: {step} Diff: {diff:.2e}".format(step=step, diff=diff))

        # Test if the difference is below our tolerance
        if diff < tolerance:
            print("Final Iteration: {step} Diff: {diff:.2e}".format(step=step, diff=diff))
            break
        
        # If we reach max_step, raise an error!
        if step == max_step:
            print("Reached max steps!")
            break

        # Update our step counter. We have a max_step because we don't want this calculation to go 
        # on forever and freeze our PC.
        step += 1
    
    # Return solved t(r)
    return tr

In [ ]:
tolerance = 1e-8
max_step = 1000
alpha = 0.2
initial_tr = np.exp(-beta * n2_ur) - 1.0

tr = Ng_iteration(tolerance, max_step, alpha, initial_tr, beta, n2_ur, n2_wk, n2_rho)

In order to do that, we need to be able to properly model the long-range electrostatics via the Coulomb potential.
A problem that arises is properly dealing with the long-range part when performing a Fourier transform - the potential will diverge!

A simple solution is to break the potential energy into two parts - a short and long range component.
Perform the fast Fourier transform for the short range component but choose the long-range component such that we can analytically transform it.

We will choose such a form in a moment, first we should define the model we want to solve.

In [ ]:
def fbt(fr, r, k, dr):
    return 2.0 * np.pi * dr * dstn(fr * r, type=4, axes=[0]) / k

def ifbt(fk, r, k, dk):
    return dk / 4.0 / np.pi / np.pi * idstn(fk * k, type=4, axes=[0]) / r

tolerance = 1e-8
max_step = 1000
alpha = 0.9
initial_tr = np.zeros(N) #np.exp(-beta * ar_ur) - 1.0

tr = Ng_iteration(tolerance, max_step, alpha, initial_tr, beta, ar_ur, ar_wk, ar_rho)